Notebook written by [Zhedong Zheng](https://github.com/zhedongzheng)

In [1]:
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import sklearn
import pprint
import itertools
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
from data import WN18

In [2]:
class Config:
    n_epochs = 30
    batch_size = 150
    embed_dim = 200

In [3]:
"""
e: entity
s: subject
p: predicate
o: object
"""
def glance_dict(d, n=5):
    return dict(itertools.islice(d.items(), n))


def read_triples(path):
    triples = []
    with open(path, 'rt') as f:
        for line in f.readlines():
            s, p, o = line.split()
            triples += [(s.strip(), p.strip(), o.strip())]
    return triples


def load_triple():
    WN18.download()
    triples_tr = read_triples('../data/WN18/wn18/train.txt')
    triples_va = read_triples('../data/WN18/wn18/valid.txt')
    triples_te = read_triples('../data/WN18/wn18/test.txt')
    triples_all = triples_tr + triples_va + triples_te
    return triples_all, triples_tr, triples_va, triples_te


def build_vocab(triples):
    params = {}
    e_set = {s for (s, p, o) in triples} | {o for (s, p, o) in triples}
    p_set = {p for (s, p, o) in triples}
    params['e_vocab_size'] = len(e_set)
    params['p_vocab_size'] = len(p_set)
    e2idx = {e: idx for idx, e in enumerate(sorted(e_set))}
    p2idx = {p: idx for idx, p in enumerate(sorted(p_set))}
    return e2idx, p2idx, params


def build_multi_label(triples_tr):
    sp2o = {}
    for (_s, _p, _o) in triples_tr:
        s, p, o = e2idx[_s], p2idx[_p], e2idx[_o] 
        if (s,p) not in sp2o:
            sp2o[(s,p)] = [o]
        else:
            if o not in sp2o[(s,p)]:
                sp2o[(s,p)].append(o)
    return sp2o


def get_y(triples_tr, e2idx, p2idx, sp2o):
    y = []
    for (_s, _p, _o) in triples_tr:
        s, p, o = e2idx[_s], p2idx[_p], e2idx[_o] 
        temp = np.zeros([len(e2idx)])
        temp[sp2o[(s,p)]] = 1.
        y.append(temp)
    y = np.asarray(y)
    return y


def next_train_batch(triples_tr, e2idx, p2idx, sp2o):
    for i in range(0, len(triples_tr), Config.batch_size):
        _triples_tr = triples_tr[i: i+Config.batch_size]
        x_s = np.asarray([e2idx[s] for (s, p, o) in _triples_tr], dtype=np.int32)
        x_p = np.asarray([p2idx[p] for (s, p, o) in _triples_tr], dtype=np.int32)
        y = get_y(_triples_tr, e2idx, p2idx, sp2o)
        yield (x_s, x_p, y)


def train_input_fn(triples_tr, e2idx, p2idx, s2p2o):
    dataset = tf.data.Dataset.from_generator(
        lambda: next_train_batch(sklearn.utils.shuffle(triples_tr),
                                 e2idx,
                                 p2idx,
                                 s2p2o),
        (tf.int32, tf.int32, tf.float32),
        (tf.TensorShape([None]),
         tf.TensorShape([None]),
         tf.TensorShape([None, len(e2idx)])))
    iterator = dataset.make_one_shot_iterator()
    x_s, x_p, y = iterator.get_next()
    return {'s': x_s, 'p': x_p}, y

In [4]:
def o_next_batch(eval_triples,
                 e2idx, 
                 p2idx):
    for (s, p, o) in tqdm(eval_triples, total=len(eval_triples), ncols=70):
        s_idx, p_idx = e2idx[s], p2idx[p]
        yield np.atleast_1d(s_idx), np.atleast_1d(p_idx)


def o_input_fn(eval_triples,
               e2idx, 
               p2idx):
    dataset = tf.data.Dataset.from_generator(
        lambda: o_next_batch(eval_triples,
                             e2idx, 
                             p2idx),
        (tf.int32, tf.int32),
        (tf.TensorShape([None,]),
         tf.TensorShape([None,])))
    iterator = dataset.make_one_shot_iterator()
    s, p = iterator.get_next()
    return {'s': s, 'p': p}


def evaluate_rank(model,
                  valid_triples,
                  test_triples,
                  all_triples,
                  e2idx,
                  p2idx):
    for eval_name, eval_triples in [('test', test_triples)]:
        _scores_o = list(model.predict(
            lambda: o_input_fn(eval_triples,
                               e2idx, 
                               p2idx)))
        ScoresO = np.reshape(_scores_o, [len(eval_triples), len(e2idx)])
        ranks_o, filtered_ranks_o = [], []
        for ((s, p, o), scores_o) in tqdm(zip(eval_triples, ScoresO),
                                          total=len(eval_triples),
                                          ncols=70):
            s_idx, p_idx, o_idx = e2idx[s], p2idx[p], e2idx[o]
            ranks_o += [1 + np.argsort(np.argsort(- scores_o))[o_idx]]
            filtered_scores_o = scores_o.copy()
            rm_idx_o = [e2idx[fo] for (fs, fp, fo) in all_triples if fs == s and fp == p and fo != o]
            filtered_scores_o[rm_idx_o] = - np.inf
            filtered_ranks_o += [1 + np.argsort(np.argsort(- filtered_scores_o))[o_idx]]
        for setting_name, setting_ranks in [('Raw', ranks_o), ('Filtered', filtered_ranks_o)]:
            mean_rank = np.mean(1 / np.asarray(setting_ranks))
            print('[{}] {} MRR: {}'.format(eval_name, setting_name, mean_rank))
            for k in [1, 3, 5, 10]:
                hits_at_k = np.mean(np.asarray(setting_ranks) <= k) * 100
                print('[{}] {} Hits@{}: {}'.format(eval_name, setting_name, k, hits_at_k))

In [5]:
def forward(features, params):
    e_embed = tf.get_variable('e_embed',
                              [params['e_vocab_size'], Config.embed_dim],
                              initializer=tf.contrib.layers.xavier_initializer())
    p_embed = tf.get_variable('p_embed',
                              [params['p_vocab_size'], Config.embed_dim],
                              initializer=tf.contrib.layers.xavier_initializer())
    
    s = tf.nn.embedding_lookup(e_embed, features['s'])
    p = tf.nn.embedding_lookup(p_embed, features['p'])
    
    logits = tf.matmul(s*p, e_embed, transpose_b=True)
    return logits
    
    
def model_fn(features, labels, mode, params):
    logits = forward(features, params)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        tf.logging.info('\n'+pprint.pformat(tf.trainable_variables()))
        tf.logging.info('params: %d'%count_train_params())
        
        loss_op = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,
                                                                        labels=labels))
        
        train_op = tf.train.AdamOptimizer().minimize(
            loss_op, global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode = mode,
                                          loss = loss_op,
                                          train_op = train_op)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions = tf.sigmoid(logits))


def count_train_params():
    return np.sum([np.prod([d.value for d in v.get_shape()]) for v in tf.trainable_variables()])


def prt_epoch(n_epoch):
    print()
    print("EPOCH %d"%(n_epoch+1))
    print()

In [6]:
triples_all, triples_tr, triples_va, triples_te = load_triple()
e2idx, p2idx, params = build_vocab(triples_all)
sp2o = build_multi_label(triples_tr)

model = tf.estimator.Estimator(model_fn,
                               params = params)

for n_epoch in range(Config.n_epochs):
    prt_epoch(n_epoch)
    model.train(lambda: train_input_fn(triples_tr, e2idx, p2idx, sp2o))
    evaluate_rank(model,
                  triples_va,
                  triples_te,
                  triples_all,
                  e2idx,
                  p2idx,)

Files Already Downloaded
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp6axh782k', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11432a6d8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}

EPOCH 1

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>]
INFO:tensorflow:params: 8192200
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow

100%|█████████████████████████████| 5000/5000 [01:53<00:00, 44.15it/s]


[test] Raw MRR: 0.0035374727540026696
[test] Raw Hits@1: 0.1
[test] Raw Hits@3: 0.16
[test] Raw Hits@5: 0.48
[test] Raw Hits@10: 1.18
[test] Filtered MRR: 0.003537649333071381
[test] Filtered Hits@1: 0.1
[test] Filtered Hits@3: 0.16
[test] Filtered Hits@5: 0.48
[test] Filtered Hits@10: 1.18

EPOCH 2

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>]
INFO:tensorflow:params: 8192200
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp6axh782k/model.ckpt-943
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 944 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp6axh782k/model.ckpt.
INFO:tensorflow:loss = 67589.39, step = 944
IN

100%|█████████████████████████████| 5000/5000 [01:52<00:00, 44.63it/s]


[test] Raw MRR: 0.01739921274610752
[test] Raw Hits@1: 1.26
[test] Raw Hits@3: 1.7000000000000002
[test] Raw Hits@5: 1.8800000000000001
[test] Raw Hits@10: 2.36
[test] Filtered MRR: 0.01764422971744893
[test] Filtered Hits@1: 1.3
[test] Filtered Hits@3: 1.7000000000000002
[test] Filtered Hits@5: 1.9
[test] Filtered Hits@10: 2.4

EPOCH 3

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>]
INFO:tensorflow:params: 8192200
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp6axh782k/model.ckpt-1886
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1887 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp6axh782k/model.ckpt.
INFO:t

...

[test] Raw MRR: 0.4866539642837297
[test] Raw Hits@1: 33.839999999999996
[test] Raw Hits@3: 58.3
[test] Raw Hits@5: 68.56
[test] Raw Hits@10: 77.24
[test] Filtered MRR: 0.6951836786751612
[test] Filtered Hits@1: 54.67999999999999
[test] Filtered Hits@3: 83.32000000000001
[test] Filtered Hits@5: 86.7
[test] Filtered Hits@10: 89.16

EPOCH 29

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>]
INFO:tensorflow:params: 8192200
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp6axh782k/model.ckpt-26404
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 26405 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp6axh782k/model.ckpt.
I

100%|█████████████████████████████| 5000/5000 [01:49<00:00, 45.70it/s]


[test] Raw MRR: 0.4872717631167406
[test] Raw Hits@1: 33.7
[test] Raw Hits@3: 58.46
[test] Raw Hits@5: 68.92
[test] Raw Hits@10: 77.48
[test] Filtered MRR: 0.6982598506047275
[test] Filtered Hits@1: 54.98
[test] Filtered Hits@3: 83.58
[test] Filtered Hits@5: 87.02
[test] Filtered Hits@10: 89.4

EPOCH 30

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>]
INFO:tensorflow:params: 8192200
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp6axh782k/model.ckpt-27347
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 27348 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp6axh782k/model.ckpt.
INFO:tensorflow:loss = 404.25214, step

100%|█████████████████████████████| 5000/5000 [01:51<00:00, 44.82it/s]


[test] Raw MRR: 0.4891431242126948
[test] Raw Hits@1: 33.82
[test] Raw Hits@3: 58.74
[test] Raw Hits@5: 69.0
[test] Raw Hits@10: 77.68
[test] Filtered MRR: 0.6980388435892407
[test] Filtered Hits@1: 54.66
[test] Filtered Hits@3: 83.94
[test] Filtered Hits@5: 87.2
[test] Filtered Hits@10: 89.7
